# Notebook to be transformed into a script

This notebook will:
- (1) Access Onedrive to get the latest money_manager file
- (2) Access onedrive to get the specified budget file 
- (3) Create a file comparing actual expenses and how they compare to the budget 

In [ ]:
# Azure connections

import logging
import os
import io
from datetime import datetime
import requests
import msal
import pandas as pd
import azure.functions as func

import yaml

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
import calendar

from datetime import datetime

# Formatting excel file 
import os
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill
from openpyxl.formatting.rule import CellIsRule

from personal_finances_utilities import generate_date_tag, load_money_manager_file


# Load the Excel file
folder_path = 'D:/GitHub/personalfinances/money_manager_data'
file_name = '2024-11-14.xlsx'
file_path = os.path.join(folder_path, file_name)

target_year = 2024



In [ ]:
expenses_df, income_df = load_money_manager_file(file_path)

In [ ]:
# Load credentials from YAML file
with open('certifications.yaml', 'r') as file:
    config = yaml.safe_load(file)

CLIENT_ID = config['client_id']
CLIENT_SECRET= config['client_secret']
TENANT_ID = config['tenant_id']
SCOPES = config['scopes']


In [ ]:
def get_access_token(client_id, tenant_id, scopes):
    authority = f"https://login.microsoftonline.com/{tenant_id}"
    
    app = msal.PublicClientApplication(
        client_id=client_id,
        authority=authority
    )
    
    # Try to acquire token silently
    accounts = app.get_accounts()
    if accounts:
        result = app.acquire_token_silent(scopes, account=accounts[0])
    else:
        result = None

    if not result:
        # Acquire token via Device Code Flow
        flow = app.initiate_device_flow(scopes=scopes)
        if not flow or 'user_code' not in flow:
            error = flow.get('error')
            error_description = flow.get('error_description')
            raise Exception(f"Failed to create device flow. Error: {error}, Description: {error_description}")
        
        print(flow['message'])  # Instructions for the user
        result = app.acquire_token_by_device_flow(flow)
    
    if 'access_token' in result:
        return result['access_token']
    else:
        error_msg = result.get('error_description', 'Unknown error')
        raise Exception(f"Could not obtain access token: {error_msg}")


In [ ]:
# Obtain access token
access_token = get_access_token(CLIENT_ID, TENANT_ID, SCOPES)

In [ ]:
# onedrive_access.py (continued)
import requests
import urllib.parse

# Set up headers
headers = {
    'Authorization': f'Bearer {access_token}'
}

# OneDrive API endpoint
#drive_api_endpoint = 'https://graph.microsoft.com/v1.0/me/drive/root/children'

subfolder_path = 'Documentos/PERSONAL FINANCES/money_manager_files'#2024-11-14.xlsx'

# URL-encode the subfolder path
encoded_subfolder_path = urllib.parse.quote(subfolder_path)
drive_api_endpoint = f'https://graph.microsoft.com/v1.0/me/drive/root:/{encoded_subfolder_path}:/children'


# Get the list of files
response = requests.get(drive_api_endpoint, headers=headers)
if response.status_code == 200:
    files_data = response.json()
    files = files_data.get('value', [])
    # Print file names
    print("Files in OneDrive Root Directory:")
    for file in files:
        print(f"{file['name']} - Last Modified: {file['lastModifiedDateTime']}")
else:
    print(f"Error retrieving files: {response.status_code}")
    print(response.json())

In [ ]:
from datetime import datetime

def get_latest_file(files):
    latest_file = None
    latest_time = None
    for file in files:
        file_time = datetime.strptime(file['lastModifiedDateTime'], '%Y-%m-%dT%H:%M:%SZ')
        if not latest_time or file_time > latest_time:
            latest_time = file_time
            latest_file = file
    return latest_file

# Find the latest expense and budget files in the subfolder
latest_expense_file = get_latest_file(files)

print("\nLatest Expense File:")
print(f"{latest_expense_file['name']} - Last Modified: {latest_expense_file['lastModifiedDateTime']}")


In [ ]:
def download_file(file, headers):
    file_id = file['id']
    download_url = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}/content"
    response = requests.get(download_url, headers=headers)
    if response.status_code == 200:
        filename = file['name']
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
        return filename
    else:
        print(f"Failed to download {file['name']}")
        return None

# Download your files
expense_file_name = download_file(latest_expense_file, headers)

In [ ]:
expenses_df = pd.read_excel(expense_file_name)

In [ ]:
expenses_df